In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.4.0'

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 7.8MB/s 
     |████████████████████████████████| 890kB 42.6MB/s 
     |████████████████████████████████| 2.9MB 24.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=781df3ec837126ebf3ddb0c0b4fefd11af7a89cf5830f7dd6c9765eccc04d8e6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

path = "/content/drive/My Drive/T-brain2/data/trainset.csv"
train_data = pd.read_csv(path)
path = "/content/drive/My Drive/T-brain2/data/testset.csv"
test_data = pd.read_csv(path)

In [ ]:
train_data.head()

,Title,Abstract,Classifications
0,"Distinctiveness, complexity, and repeatability...",This paper proposes three measures to quantify...,ENGINEERING
1,An Adaptable Fast Matrix Multiplication Algori...,In this paper we present an adaptable fast mat...,EMPIRICAL
2,Trimming the Multipath for Efficient Dynamic R...,Multipath routing is a trivial way to exploit ...,THEORETICAL ENGINEERING
3,Learning Word Embeddings from Speech,"In this paper, we propose a novel deep neural ...",ENGINEERING EMPIRICAL
4,A Unified Model for Near and Remote Sensing,We propose a novel convolutional neural networ...,THEORETICAL ENGINEERING


In [ ]:
train_data["Classifications"].value_counts()

THEORETICAL                          1964
ENGINEERING                          1902
EMPIRICAL                            1009
THEORETICAL ENGINEERING               736
ENGINEERING EMPIRICAL                 613
THEORETICAL EMPIRICAL                 375
OTHERS                                259
THEORETICAL ENGINEERING EMPIRICAL     142
Name: Classifications, dtype: int64

In [ ]:
replace_dict={"THEORETICAL":1,
"ENGINEERING":2,
"EMPIRICAL":3,
"THEORETICAL ENGINEERING":4,
"ENGINEERING EMPIRICAL":5,
"THEORETICAL EMPIRICAL":6,
"OTHERS":7,
"THEORETICAL ENGINEERING EMPIRICAL":8}

In [ ]:
train_data["Classifications"]=train_data["Classifications"].map(replace_dict)

In [ ]:
test_data.head()

,Id,Title,Abstract
0,1,Efficient Visual Coding: From Retina To V2,The human visual system has a hierarchical str...
1,2,PolyMorph: Increasing P300 Spelling Efficiency...,P300 is an electric signal emitted by brain ab...
2,3,"Cooperative Cognitive Networks: Optimal, Distr...",This paper considers the cooperation between a...
3,4,Indoor occupancy estimation from carbon dioxid...,This paper presents an indoor occupancy estima...
4,5,Lower Bound for General Circuits Computing Cli...,We prove an exponential lower bound for genera...


In [ ]:
train_data["Classifications"]

0       2
1       3
2       4
3       5
4       4
       ..
6995    6
6996    2
6997    3
6998    3
6999    1
Name: Classifications, Length: 7000, dtype: int64

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
max_length=32

In [ ]:
import numpy as np
x_input=np.empty((0,max_length),int)
for doc in train_data["Title"]:
  temp=tokenizer(train_data["Title"][0],max_length = max_length,padding="max_length",truncation=True)
  x_input=np.append(x_input,[temp['input_ids']],axis=0)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sample_method=RandomUnderSampler(random_state=143)
X_train,y_train=sample_method.fit_sample(x_input,np.array(train_data["Classifications"]))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
np.unique(y_train,return_counts=1)

(array([1, 2, 3, 4, 5, 6, 7, 8]),
 array([142, 142, 142, 142, 142, 142, 142, 142]))

In [ ]:
from transformers import TFRobertaForSequenceClassification
import tensorflow as tf
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 8)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-05)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
model.compile(optimizer=optimizer,loss=loss,metrics=[metric])

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "tf_roberta_for_sequence_classification_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124055040 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  596744    
Total params: 124,651,784
Trainable params: 124,651,784
Non-trainable params: 0
_________________________________________________________________


In [ ]:
roberta_history = model.fit(X_train,y_train,
                            epochs=30,batch_size=16,shuffle=False,verbose=1,validation_split=0.4)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/30


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


43/43 [==============================] - ETA: 0s - loss: 1.9598 - accuracy: 0.3632

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


43/43 [==============================] - 24s 229ms/step - loss: 1.9621 - accuracy: 0.3590 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/30
43/43 [==============================] - 7s 170ms/step - loss: 2.0568 - accuracy: 0.1647 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/30
43/43 [==============================] - 7s 169ms/step - loss: 1.9426 - accuracy: 0.2133 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/30
43/43 [==============================] - 7s 169ms/step - loss: 1.8734 - accuracy: 0.1347 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/30
43/43 [==============================] - 7s 169ms/step - loss: 1.8616 - accuracy: 0.0934 - val_loss: nan - val_accuracy: 0.0637
Epoch 6/30
43/43 [==============================] - 7s 170ms/step - loss: 1.8341 - accuracy: 0.1665 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/30
43/43 [==============================] - 7s 170ms/step - loss: 1.7131 - accuracy: 0.2074 - val_loss: nan - val_accuracy: 0.0637
Epoch 8/30
43/43 